In [2]:
from architector import view_structures,convert_io_molecule, CalcExecutor
from executorlib import SingleNodeExecutor
import pandas as pd

In [3]:
def relax_sample_xtb(inlst):
    import os
    os.environ['UMA_MODEL_PATH'] = ''
    os.environ['OMOL_MODEL_PATH'] = ''
    
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    os.environ["MKL_NUM_THREADS"] = "1"
    os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
    os.environ["NUMEXPR_NUM_THREADS"] = "1"
    
    from architector import CalcExecutor, convert_io_molecule
    struct = inlst[0]
    name = inlst[1]
    mol = convert_io_molecule(struct)
    try:
        out = CalcExecutor(struct,relax=True,save_trajectories=True)
    except:
        out = None
    returns = []
    if out is not None:
        if out.trajectory is not None:
            if len(out.trajectory) > 0:
                sn = int(len(out.trajectory)/4)
                if sn < 1:
                    sn = 1
                samples = out.trajectory[::sn]
                for i,thing in enumerate(samples):
                    mol.ase_atoms = thing
                    returns.append(mol.write_mol2(name+'{}'.format(i),
                                                  writestring=True))
            else:
                returns.append('None - traj len 0')
        else:
            returns.append('None - None traj')
    else:
        returns.append('None - CalcExe failed')
    return returns

In [4]:
out = CalcExecutor('CCC',relax=True)
out.trajectory is not None

False

In [5]:
indf = pd.read_csv('all_preliminary_clusters.csv')

In [6]:
from concurrent.futures import as_completed
from tqdm import tqdm
done_list = []
with SingleNodeExecutor(max_workers=100, block_allocation=True) as exe:
    print('submitting')
    futs = []
    for i,row in indf.iterrows():
        inlst = [row['struct'],row['uid']]
        fut = exe.submit(relax_sample_xtb, inlst)
        futs.append(fut)
    with tqdm(total=len(futs)) as pbar:
        for done in as_completed(futs):
            done_list.append(done.result())
            pbar.update(1)

submitting


100%|██████████| 32580/32580 [31:34<00:00, 17.19it/s] 


In [7]:
len(done_list)

32580

In [8]:
flat_list = [item for sublist in done_list for item in sublist]

In [9]:
non_not = [x for x in flat_list if 'None - ' not in x]

In [10]:
len(non_not)

135227

In [11]:
len(flat_list)

138639

In [12]:
outrows = []
for thing in tqdm(non_not,total=len(non_not)):
    mol = convert_io_molecule(thing)
    td = dict()
    td['name'] = thing.splitlines()[1].split()[0]
    td['mol2'] = thing
    td['charge'] = mol.charge
    td['mult'] = mol.uhf + 1
    td['xyz'] = mol.write_xyz(td['name'],writestring=True)
    outrows.append(td)

100%|██████████| 135227/135227 [01:17<00:00, 1752.77it/s]


In [13]:
odf = pd.DataFrame(outrows)

In [14]:
odf.iloc[0]

name                                   hexagonal_planarSi60
mol2      @<TRIPOS>MOLECULE\nhexagonal_planarSi60 Charge...
charge                                                  0.0
mult                                                      1
xyz       6\n\nSi -0.7077 1.6645 1.1576\nSi 0.7077 -1.66...
Name: 0, dtype: object

In [15]:
odf.to_csv('all_structs_sampled.csv',index=False)

In [17]:
!du -sh all_structs_sampled.csv

215M	all_structs_sampled.csv


In [21]:
view_structures(odf.iloc[20000:20012].xyz)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [19]:
odf.shape

(135227, 5)

In [24]:
subset = odf[['name','charge','mult','xyz']]

In [26]:
subset['charge'] = [int(x) for x in subset.charge.values]

In [27]:
subset.to_csv('only_name_charge_mult_xyz.csv',index=False)

In [28]:
!du -sh only_name_charge_mult_xyz.csv

37M	only_name_charge_mult_xyz.csv
